In [16]:
import os
import platform

import openai
import chromadb
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader

In [17]:
loader = PyPDFLoader("crm.pdf")
pages = loader.load_and_split()

In [24]:
os.environ["OPENAI_API_KEY"] = ''
openai.api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

In [19]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(pages, embedding=embeddings, persist_directory=".")
vectorstore.persist()

Using embedded DuckDB with persistence: data will be stored in: .


In [21]:
pdf_qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [32]:
query = """与えられた文章を以下のフォーマットで日本語で出力してください。```

・概要

・技術的なハイライト
　・ハイライト１
　・ハイライト２
　・ハイライト３

・分類キーワード
　・キーワード１
　・キーワード２
　・キーワード３
```"""
chat_history = []

result = pdf_qa({"question": query, "chat_history": chat_history})

result['answer']

'概要：\nこの文章はCRMに関する技術的な情報を含む文章です。CRMの概要や、CRMとIT、マーケティングとの関係、そしてCRMに関するハイライトについて言及しています。また、この文章には1948年から2004年までの時系列情報が含まれています。\n\n技術的なハイライト：\n・CRMシステム\n・CRMの商業利用\n・マーケティングオートメーション\n\n分類キーワード：\n・CRM\n・マーケティング\n・IT'